# Haram Blur using Different Models and Approaches  
> 


### ✅ 1.(Detetctor = OpenCV) + (Gender Model =  Caffe Model)

**OpenCV Haar est dépassé et moins précis, à éviter pour un projet sérieux**.

#### 1.1 Try Model on Images 

In [25]:
import cv2

# Import Image to test model on 
img = cv2.imread('images.jpg')
print(f"Image Shape: {img.shape}")
# resize the image
img = cv2.resize(img,(302,215))

# Define the Classifier To detect the Faces and return => (x,y,w,h)
cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = cascade.detectMultiScale(img , 1.1 , 11)

# Load the Model
modelFile = "gender_net.caffemodel"
configFile = "gender_deploy.prototxt"
net = cv2.dnn.readNet(modelFile, configFile)
genderList = ['Male', 'Female']

for i,(x,y,w,h) in enumerate(faces):
    roi = img[y-5:y+h+5 , x-5:x+w+5]

    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    blob = cv2.dnn.blobFromImage(roi, 1, (227,227), (78.4263377603, 87.7689143744, 114.895847746))
    net.setInput(blob)  
    genderPreds = net.forward()
    gender = genderList[genderPreds[0].argmax()]
    confidence = f'{genderPreds[0].max():.2f}'
    print(f'Gender: {gender} with a confidence of {confidence}')
    if gender == 'Female':
        img[y-5:y+h+5 , x-5:x+w+5] = cv2.GaussianBlur(roi,(99,99),33)
    cv2.putText(img,gender,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,255),2,cv2.LINE_AA)

    

cv2.imshow("boyand girl",img)
cv2.imwrite('HaramBlur.jpg',img)
if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()

Image Shape: (185, 272, 3)
Gender: Female with a confidence of 1.00
Gender: Female with a confidence of 0.53


#### 1.2 Try the model on Real Time Images From Webcam

In [13]:
import cv2
import cvzone
cap = cv2.VideoCapture(0)
colors ={'pink':(255,0,255),'green' :(0,255,0) }

while True:
    # Capture the Frame to Pass to The model =>Detetct How Many Faces There
    sucess , img = cap.read()
    faces = cascade.detectMultiScale(img , 1.1 , 11)

    # For Each Boundiing Box Detetcted => Run The Model
    for (x,y,w,h) in faces:
        # Get the ROI : Where the Face is Located AND Detetct it 

        roi = img[max(0,y-5):y+h+5 , max(0,x-5):x+w+5]
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,255),2)

        # Run The Model
        blob = cv2.dnn.blobFromImage(roi, 1, (227,227), (78.4263377603, 87.7689143744, 114.895847746))
        net.setInput(blob)  
        genderPreds = net.forward()
        gender = genderList[genderPreds[0].argmax()]
        confidence = f'{genderPreds[0].max():.2f}'
        message = f'Gender: {gender} with a confidence of {confidence}'
        if gender == 'Female':
            cvzone.putTextRect(img,message,(x,y-10),colorR=colors['pink'],colorB=(0,0,0),scale=1,thickness=2)
            img[y-5:y+h+5 , x-5:x+w+5] = cv2.GaussianBlur(roi,(99,99),33)
        else : 
            cvzone.putTextRect(img,message,(x,y-10),colorR=colors['green'],colorB=(0,0,0),scale=1,thickness=2)
        
    cv2.imshow("Gender Detection",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            cap.release()
            break    

### ✅ 2.(Detetctor = Mediapipe) + (Gender Model =  Caffe Model)


#### 2.1 Initailize The Lightweight model

In [18]:
cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
modelFile = "gender_net.caffemodel"
configFile = "gender_deploy.prototxt"
net = cv2.dnn.readNet(modelFile, configFile)
genderList = ['Male', 'Female']
colors ={'pink':(255,0,255),'green' :(0,255,0) }


In [24]:
import cv2
import mediapipe as mp
import cvzone

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image_rgb)

        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = image.shape
                x = int(bbox.xmin * w)
                y = int(bbox.ymin * h)
                width = int(bbox.width * w)
                height = int(bbox.height * h)

                face = image[y:y+height, x:x+width]
                # Detect if the face is a women or a Men 
                blob = cv2.dnn.blobFromImage(face,1,(227,227),(78.4263377603, 87.7689143744, 114.895847746))
                net.setInput(blob)
                genderPreds = net.forward()
                gender = genderList[genderPreds[0].argmax()]


                if gender == 'Female':
                    image[y:y+height, x:x+width] = cv2.GaussianBlur(face, (99, 99), 30)
                    cv2.rectangle(image, (x, y), (x+width, y+height), color=colors["pink"], thickness=2)
                    cvzone.putTextRect(image,f'Gender: {gender}',(x,y-20),2,2,colorR=colors['pink'])
                else :  
                    cvzone.cornerRect(image,(x,y,width,height),colorR=colors['green'])
                    cvzone.putTextRect(image,f'Gender: {gender}',(x,y-20),2,2,colorR=colors['green'])

        cv2.imshow('Haram Blur', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()
